# AB Test - Marketing Campaign Analysis

Now, I'll conduct the A/B Test on our marketing campaign dataframe. It's important to stabilish:
- I'm analysing the promotions financial performance, so my MAIN METRIC will be `SalesInThousands`
- My hyphotesis are:
    - H0: the promotions did not have effect on revenue
    - H1: the promotions HAD an effect on revenue

Before begin doing test, IT'S FUNDAMENTAL TO LOOK INTO THE TYPE OF DISTRIBUTION I HAVE. For this, I'll use Shapiro Normality test and then determine which statiscial metrics I'll use.

In [1]:
import pandas as pd
from scipy.stats import shapiro, ttest_1samp, ttest_ind, levene, kruskal, mannwhitneyu, pearsonr, spearmanr

In [2]:
# import dataframe
df = pd.read_csv('WA_Marketing-Campaign.csv')

# keep only the metric we'll use to analyse promotion performance: SalesInThousands
df_clean = df[['Promotion', 'SalesInThousands']]

## Normality Check

In [3]:
# check to see if df has a normal distribution
## H0: df HAS a normal distribution: I'll move to parametric methods
## H1: df DOESN'T have a normal distribution: I'll use NON parametric methods

# Shapiro test is done in arrays, so I need to create an array for each promotion
promo_list = df_clean['Promotion'].unique()

# loop shapiro test between promotion list(1,2,3) and extract the p-value
for promotion in promo_list:
    normal_test = shapiro(df_clean.loc[df_clean['Promotion'] == promotion, 'SalesInThousands'])
    if normal_test[1] < 0.05:
        print(f'Promotion {promotion} DOESN NOT HAVE a normal distribution \n')
    else:
        print(f'Promotion {promotion} HAS a normal distribution \n')

Promotion 3 DOESN NOT HAVE a normal distribution 

Promotion 2 DOESN NOT HAVE a normal distribution 

Promotion 1 DOESN NOT HAVE a normal distribution 



Since our promotion don't have a normal distribution (AS NOTICED DURING THE EDA), I'll use **Nonparametric** statistical methods to determine if there's any didfference between the promotions.

## A/B Test

### Levene Test - Variance
HO: samples have same variance

H1: samples DO NOT HAVE the same variance

In [4]:
# Leven test compares samples in the format of arrays
## Since I have 3 samples, because I have 3 promotion, I need compare them all at the same time
levene_test, levene_pvalue = levene(df_clean.loc[df_clean['Promotion'] == 1, 'SalesInThousands'],
                                    df_clean.loc[df_clean['Promotion'] == 2, 'SalesInThousands'],
                                    df_clean.loc[df_clean['Promotion'] == 3, 'SalesInThousands'])

if levene_pvalue < 0.05:
    print(f'Promotions have different variance, P-value={round(levene_pvalue,4)} < 0.05')
else:
    print(f'Promotions have the same variance, P-value={round(levene_pvalue,4)} > 0.05')

Promotions have the same variance, P-value=0.2818 > 0.05


### Kruskal-Wallis H-test - Median
HO: sample medians are the same

H1: sample median are different

In [5]:
kruskal_test, kruskal_pvalue = kruskal(df_clean.loc[df_clean['Promotion'] == 1, 'SalesInThousands'],
                                    df_clean.loc[df_clean['Promotion'] == 2, 'SalesInThousands'],
                                    df_clean.loc[df_clean['Promotion'] == 3, 'SalesInThousands'])

if kruskal_pvalue < 0.05:
    print(f'Promotions have DIFFERENT medians, P-value={round(kruskal_pvalue,4)} < 0.05')
else:
    print(f'Promotions have the SAME median, P-value={round(kruskal_pvalue,4)} > 0.05')

Promotions have DIFFERENT medians, P-value=0.0 < 0.05


Comparing the 3 promotions, I identified there's a difference between them (due to the different median) and it's statistical significant. The problem I can't pinpoint which one is different, since my **variance** analysis suggest they are the same.

To fix this, I'll have to run individual tests and see which ones are different:
- Promotion 1 x Promotion 2
- Promotion 1 x Promotion 3
- Promotion 2 x Promotion 3

For this, I'm gonna conduct the a Dunn's Test that is usually conducted after an inconclusive Kurskall-Wallis Test.

In [6]:
!pip install scikit_posthocs

In [10]:
import scikit_posthocs as sp

# Dunn will determine if there's a difference between the promotions by investigating the pairs
posthoc_results = sp.posthoc_dunn([df_clean[df_clean["Promotion"] == p]["SalesInThousands"] for p in df_clean["Promotion"].unique()], p_adjust='bonferroni') # to avoid randomness, I'll add Bonferroni test

# Print the comparison between all (1x2, 1x3,2x3)
## H0: there is no difference between A and B
## H1: there IS a difference between A and B
print(round(posthoc_results,4))


        1    2       3
1  1.0000  0.0  0.1459
2  0.0000  1.0  0.0000
3  0.1459  0.0  1.0000


## A/B TEST Summary

In this project, I ran an A/B Test to evaluate the effectiveness of 3 marketing campaigns for a fast food company. Based on the statistical analysis, promotion 1 and 2 are statistical different from each other. Promotion 3 on the other hand, is  different from promotion 2 but similar to promotio 1.

Insights BASED ONLY ON THE AVAILABLE INFORMATION:
- Based on EDA analysis, the company should chose to move with Promotions 1 OR 3
- My recommendation is to move with Promotion 1, despite the lower volume in sales, Promotion 1 has the highest avarage revenue
- Drop promotion 2, despite have a higher sales volume, it has the lowest revenue